In [150]:
import os
import time
import json
import re
import pprint
import csv
import time
import xml.etree.ElementTree as ET

In [151]:
# regex objects
proxy_re = re.compile('proxy\.bc\.edu\/login\?url=', re.IGNORECASE)
remove_protocol_re = re.compile('https?:\/\/', re.IGNORECASE)
domain_name_re = re.compile('(www\.)?(?P<dn>[a-zA-Z0-9.\-_]*).*', re.IGNORECASE)

In [152]:
# load xml dump and get root element
db_dump_file = 'libguides-export_1520553858.xml'

# get filename without extension
db_dump_file_no_extention = re.sub('\.xml', '', db_dump_file)

tree = ET.parse(db_dump_file)
root = tree.getroot()

In [153]:
# test that import worked
root

<Element 'libguides' at 0x13bd99688>

In [154]:
# get total number of accounts
total_accounts = len(root.findall('./accounts/account'))
total_accounts

100

In [155]:
# get total number of guides
total_guides = len(root.findall('./guides/guide'))
total_guides

813

In [156]:
# get total number of pages
total_pages = len(root.findall('.//pages/page'))
total_pages

4648

In [157]:
# get total number of boxes
total_boxes = len(root.findall('.//boxes/box'))
total_boxes

13346

In [158]:
# get total number of assets
total_assets = len(root.findall('.//assets/asset'))
total_assets

29952

In [159]:
# get total number of link assets
link_count = 0
for link in root.findall('.//assets/asset'):
    if link.findtext('type') == "Link":
        link_count += 1
        
link_count

14657

In [160]:
# get all guide authors and add them to accounts_dict dictionary
accounts_count = 0
accounts_dict = {}
for account in root.findall('./accounts/account'):
    acct_id = account.findtext('id')
    acct_fname = account.findtext('first_name')
    acct_lname = account.findtext('last_name')
    accounts_dict[acct_id] = "%s %s" % (acct_fname, acct_lname)
    accounts_count += 1
    
accounts_count

100

In [161]:
# test output
accounts_dict['10678']

'Leslie Homzie'

In [162]:
# load list of known database domain names
db_domain_names = [
    '18thcjournals.amdigital.co.uk',
    'aadr.alexanderstreet.com',
    'aapredbook.org',
    'abcas3.auditedmedia.com',
    'academic.marketresearch.com',
    'academic.mintel.com',
    'access.newspaperarchive.com',
    'access.vault.com',
    'accessible-archives.com',
    'accessible.com',
    'accessscience.com',
    'acta.chadwyck.com',
    'afi.chadwyck.com',
    'airitilibrary.com',
    'alacra.com',
    'amadeus.bvdinfo.com',
    'analyticscampus.gallup.com',
    'anb.org',
    'ancestrylibrary.proquest.com',
    'anthrosource.onlinelibrary.wiley.com',
    'apps.brepolis.net',
    'apps.intelligize.com',
    'archive.irishnewsarchive.com',
    'archives.chadwyck.com',
    'archivesdirect.amdigital.co.uk',
    'artfl-project.uchicago.edu',
    'askart.com',
    'asp6new.alexanderstreet.com',
    'atho.alexanderstreet.com',
    'balkaninsight.com',
    'bap.chadwyck.com',
    'bc.app.movie-discovery.com',
    'bc.kanopystreaming.com',
    'bc.policymap.com',
    'bdsl-online.de',
    'beckettarchive.org',
    'bibliorossica.com',
    'biocyc.org',
    'blackwellreference.com',
    'bldr.alexanderstreet.com',
    'bltc.alexanderstreet.com',
    'blww.alexanderstreet.com',
    'booksinprint.com',
    'bostoncollege.naxosmusiclibrary.com',
    'brecht.chadwyck.com',
    'brepolis.net',
    'brepolsonline.net',
    'bridgemaneducation.com',
    'britishonlinearchives.co.uk',
    'bsc.chadwyck.com',
    'buddhism-dict.net',
    'bwld.alexanderstreet.com',
    'cabells.com',
    'cali.alexanderstreet.com',
    'cepr.org',
    'chicagomanualofstyle.org',
    'china.amdigital.co.uk',
    'churchillarchive.com',
    'ciaonet.org',
    'cios.org',
    'classic.accountingresearchmanager.com',
    'classiques-garnier.com',
    'clcd.com',
    'clients1.ibisworld.com',
    'clt.brepolis.net',
    'cochranelibrary.com',
    'cognet.mit.edu',
    'collections.chadwyck.com',
    'collegesource.org',
    'colonial.chadwyck.com',
    'colonialamerica.amdigital.co.uk',
    'communicationencyclopedia.com',
    'computingreviews.com',
    'conference-board.org',
    'congressional.proquest.com',
    'connect.jbiconnectplus.org',
    'corporateaffiliations.com',
    'cpps.brepolis.net',
    'crl.edu',
    'cwld.alexanderstreet.com',
    'daredictionary.com',
    'databank.worldbank.org',
    'dbpo.chadwyck.com',
    'degruyter.com',
    'demographics.geolytics.com',
    'dgr.rints.com',
    'dib.cambridge.org',
    'dictionaries.brillonline.com',
    'digitalmicrofilm.proquest.com',
    'digitaltheatreplus.com',
    'digizeitschriften.de',
    'dl.acm.org',
    'dlcr.alexanderstreet.com',
    'dlib.eastview.com',
    'dramaonlinelibrary.com',
    'duxiu.com',
    'e-enlightenment.com',
    'earlyrepublic.press.jhu.edu',
    'eas-ref.press.jhu.edu',
    'eastview.com',
    'ebooks.cambridge.org',
    'edu.adspender.kantarmediana.com',
    'edu.medici.tv',
    'eduref.org',
    'eebo.chadwyck.com',
    'eena.alexanderstreet.com',
    'efts.lib.uchicago.edu',
    'elibrary.worldbank.org',
    'embase.com',
    'emeraldinsight.com',
    'emis.com',
    'empire.amdigital.co.uk',
    'englishhistoricaldocuments.com',
    'esgdirect.msci.com',
    'estc.bl.uk',
    'ethnologue.com',
    'ets.umdl.umich.edu',
    'europeanamericana.com',
    'fconline.foundationcenter.org',
    'fiaf.chadwyck.com',
    'fii.chadwyck.com',
    'find.galegroup.com',
    'fod.infobase.com',
    'foreignaffairs.com',
    'foreignlawguide.com',
    'galenet.galegroup.com',
    'gateway.ovid.com',
    'gateway.proquest.com',
    'gdc.galegroup.com',
    'geoscienceworld.org',
    'gerritsen.chadwyck.com',
    'global.factiva.com',
    'globalcommodities.amdigital.co.uk',
    'glossary.agiweb.org',
    'go.galegroup.com',
    'goethe.chadwyck.com',
    'gtionline.foundationcenter.org',
    'guidestar.org',
    'hanchi.ihp.sinica.edu.tw',
    'hapi.ucla.edu',
    'hbcponline.com',
    'heinonline.org',
    'history.paratext.com',
    'hstalks.com',
    'hsus.cambridge.org',
    'humanities.uchicago.edu',
    'ic.galegroup.com',
    'ica.princeton.edu',
    'icpsr.umich.edu',
    'icsd.fiz-karlsruhe.de',
    'ieee.org',
    'ieeexplore.ieee.org',
    'imld.alexanderstreet.com',
    'infotrac.galegroup.com',
    'infoweb.newsbank.com',
    'ingentaconnect.com',
    'iop.org',
    'irishbooksearch.com',
    'isiknowledge.com',
    'issgovernance.com',
    'iwrp.alexanderstreet.com',
    'jcr.incites.thomsonreuters.com',
    'jstor.org',
    'kafka.chadwyck.com',
    'kll-online.de',
    'kluwerlawonline.com',
    'knovel.com',
    'knowledge.sagepub.com',
    'latinnews.com',
    'laww.alexanderstreet.com',
    'lb.brillonline.nl',
    'leme.library.utoronto.ca',
    'lib.uchicago.edu',
    'library.artstor.org',
    'library.cqpress.com',
    'library.nlx.com',
    'link.springer.com',
    'literature.proquest.com',
    'litguide.press.jhu.edu',
    'llmcdigital.org',
    'lobbyists.info',
    'loebclassics.com',
    'luther.chadwyck.com',
    'lyellcollection.org',
    'massobservation.amdigital.co.uk',
    'materials.springer.com',
    'mathscinet.ams.org',
    'medievalfamilylife.amdigital.co.uk',
    'medievaltravel.amdigital.co.uk',
    'mergentarchives.com',
    'mergentintellect.com',
    'mergentonline.com',
    'metasearch.opinionarchives.com',
    'methods.sagepub.com',
    'metopera.org',
    'microform.digital',
    'mio.asminternational.org',
    'moneyline.cq.com',
    'monthlycatalog.chadwyck.com',
    'morganclaypool.com',
    'moto.alexanderstreet.com',
    'mrw.interscience.wiley.com',
    'muse.jhu.edu',
    'myinsight.globalinsight.com',
    'na.capitaliq.com',
    'nano.nature.com',
    'nature.com',
    'ncbi.nlm.nih.gov',
    'newfirstsearch.oclc.org',
    'news.siteintelgroup.com',
    'nexisuni.com',
    'next.srds.com',
    'ntrl.ntis.gov',
    'nursinglibrary.org',
    'nwld.alexanderstreet.com',
    'nybooks.com',
    'oecd-ilibrary.org',
    'oed.com',
    'online.epocrates.com',
    'online.goinglobal.com',
    'online.ibfd.org',
    'online.infobaselearning.com',
    'online.liverpooluniversitypress.co.uk',
    'onlinelibrary.wiley.com',
    'osapublishing.org',
    'oversea.cnki.net',
    'ovidsp.ovid.com',
    'oxfordaasc.com',
    'oxfordartonline.com',
    'oxfordbiblicalstudies.com',
    'oxfordbibliographies.com',
    'oxforddictionaries.com',
    'oxforddnb.com',
    'oxfordhandbooks.com',
    'oxfordislamicstudies.com',
    'oxfordmusiconline.com',
    'oxfordreference.com',
    'oxfordscholarlyeditions.com',
    'palgraveconnect.com',
    'papers.nber.org',
    'parkerweb.stanford.edu',
    'parlipapers.proquest.com',
    'pep-web.org',
    'perditamanuscripts.amdigital.co.uk',
    'perseus.tufts.edu',
    'phaseonline.ceramics.org',
    'philpapers.org',
    'pld.chadwyck.com',
    'pm.nlx.com',
    'poll.orspub.com',
    'primarysources.brillonline.com',
    'projecteuclid.org',
    'proquestcombo.safaribooksonline.com',
    'psychiatryonline.com',
    'pubs.acs.org',
    'pubs.rsc.org',
    'purl.org',
    'quod.lib.umich.edu',
    'readyreference.data-planet.com',
    'reaxys.com',
    'reference.paratext.com',
    'referenceusa.com',
    'referenceworks.brillonline.com',
    'religion.oxfordre.com',
    'rep.routledge.com',
    'research.valueline.com',
    'responsa.co.il',
    'rkma.com',
    'rocksbackpages.com',
    'romanticism.amdigital.co.uk',
    'ropercenter.cornell.edu',
    'rotunda.upress.virginia.edu',
    'ruhr-uni-bochum.de',
    'schiller.chadwyck.com',
    'school.eb.com',
    'sciencedirect.com',
    'scitation.org',
    'scopus.com',
    'search.alexanderstreet.com',
    'search.eb.com',
    'search.ebscohost.com',
    'search.epnet.com',
    'search.itergateway.org',
    'search.marquiswhoswho.com',
    'search.proquest.com',
    'shmu.alexanderstreet.com',
    'sk.sagepub.com',
    'snl.com',
    'socialexplorer.com',
    'socialismonfilm.amdigital.co.uk',
    'solomon.dkbl.alexanderstreet.com',
    'solomon.tcpt.alexanderstreet.com',
    'spiedigitallibrary.org',
    'springerlink.com',
    'stalindigitalarchive.com',
    'stat.unido.org',
    'statabs.proquest.com',
    'statcan.gc.ca',
    'statehousenews.com',
    'statindex.org',
    'statista.com',
    'statistical.proquest.com',
    'statisticaldatasets.data-planet.com',
    'stephanus.tlg.uci.edu',
    'swrp.alexanderstreet.com',
    'syrano.demarque.com',
    'tandfonline.com',
    'tapor.library.utoronto.ca',
    'teachingbooks.net',
    'teso.chadwyck.com',
    'tol.org',
    'totalaccess.emarketer.com',
    'trackingterrorism.org',
    'tradelawguide.com',
    'trials.proquest.com',
    'tvnews.vanderbilt.edu',
    'ulrichsweb.com',
    'universitypublishingonline.org',
    'victorianperiodicals.com',
    'voices.revealdigital.com',
    'web.lexis-nexis.com',
    'webcsd.ccdc.cam.ac.uk',
    'webofknowledge.com',
    'wellesley.chadwyck.com',
    'wisertrade.org',
    'wkcheetah.com',
    'wnc.eastview.com',
    'worldchristiandatabase.org',
    'worldreligiondatabase.org',
    'worldshakesbib.org',
    'worldtradelaw.net',
    'wwp.northeastern.edu',
    'www3.interscience.wiley.com',
    'ybio.brillonline.com',
    'zbmath.org'
]
db_domain_names_set = set(db_domain_names)

print(len(db_domain_names_set))

335


In [163]:
db_names = [
    'aadr.alexanderstreet.com',
    'abcas3.auditedmedia.com/micenter/micenter',
    'academic.mintel.com',
    'access.newspaperarchive.com',
    'access.vault.com/career-insider-login.aspx?aid=148188',
    'acta.chadwyck.com',
    'afi.chadwyck.com',
    'amadeus.bvdinfo.com/ip',
    'analyticscampus.gallup.com/?ref=auth',
    'ancestrylibrary.proquest.com',
    'anthrosource.onlinelibrary.wiley.com',
    'apps.brepolis.net/brepolisportal/default.aspx',
    'apps.brepolis.net/dhge/test/default2.aspx',
    'apps.brepolis.net/lexiema/test/default2.aspx',
    'apps.intelligize.com',
    'archive.irishnewsarchive.com/olive/apa/ina.edu/default.aspx#panel=home',
    'archives.chadwyck.com',
    'artfl-project.uchicago.edu/content/tfa',
    'asp6new.alexanderstreet.com/orhi',
    'asp6new.alexanderstreet.com/sixt',
    'asp6new.alexanderstreet.com/was2/was2.index.map.aspx',
    'atho.alexanderstreet.com',
    'bap.chadwyck.com',
    'bc.app.movie-discovery.com',
    'bc.kanopystreaming.com',
    'bc.policymap.com/maps',
    'bldr.alexanderstreet.com',
    'bltc.alexanderstreet.com',
    'blww.alexanderstreet.com',
    'bostoncollege.naxosmusiclibrary.com',
    'bostoncollege.naxosmusiclibrary.com/world',
    'brecht.chadwyck.com/deutsch/frames/home/home',
    'bsc.chadwyck.com',
    'bwld.alexanderstreet.com',
    'cali.alexanderstreet.com',
    'classic.accountingresearchmanager.com/wk/rm.nsf/arm.html?open',
    'clients1.ibisworld.com',
    'clt.brepolis.net/acll/default.aspx',
    'clt.brepolis.net/dld/default.aspx',
    'clt.brepolis.net/emgh/pages/search.aspx',
    'cognet.mit.edu',
    'collections.chadwyck.com/bie/htxview?template=basic.htx&content=frameset.htx',
    'collections.chadwyck.com/eas/htxview?template=basic.htx&content=frameset.htx',
    'collections.chadwyck.com/home/home_20aap.jsp',
    'collections.chadwyck.com/home/home_20ap.jsp',
    'collections.chadwyck.com/home/home_20ep.jsp',
    'collections.chadwyck.com/home/home_aap.jsp',
    'collections.chadwyck.com/home/home_abell.jsp',
    'collections.chadwyck.com/home/home_ad.jsp',
    'collections.chadwyck.com/home/home_ap.jsp',
    'collections.chadwyck.com/home/home_bal.jsp',
    'collections.chadwyck.com/home/home_c18f.jsp',
    'collections.chadwyck.com/home/home_c19f.jsp',
    'collections.chadwyck.com/home/home_cp.jsp',
    'collections.chadwyck.com/home/home_eaf2.jsp',
    'collections.chadwyck.com/home/home_ed.jsp',
    'collections.chadwyck.com/home/home_eepf.jsp',
    'collections.chadwyck.com/home/home_ep.jsp',
    'collections.chadwyck.com/home/home_ep2.jsp',
    'collections.chadwyck.com/home/home_fpl.jsp',
    'collections.chadwyck.com/home/home_kjb.jsp',
    'collections.chadwyck.com/yeats/htxview?template=basic.htx&content=frameset.htx',
    'colonial.chadwyck.com',
    'congressional.proquest.com/congressional/search/advanced/advanced?accountid=9673',
    'connect.jbiconnectplus.org',
    'cpps.brepolis.net/aph/search.cfm',
    'cpps.brepolis.net/bbih/search.cfm?',
    'cpps.brepolis.net/bmb/search.cfm',
    'cwld.alexanderstreet.com',
    'databank.worldbank.org/data/views/variableselection/selectvariables.aspx?source=education-statistics',
    'databank.worldbank.org/data/views/variableselection/selectvariables.aspx?source=gender-statistics',
    'databank.worldbank.org/data/views/variableselection/selectvariables.aspx?source=health-nutrition-and-population-statistics',
    'databank.worldbank.org/data/views/variableselection/selectvariables.aspx?source=world-development-indicators',
    'dbpo.chadwyck.com',
    'demographics.geolytics.com/ncdb2010/default.aspx',
    'demographics.geolytics.com/sf32000/preloginform.aspx',
    'dgr.rints.com',
    'dib.cambridge.org',
    'dictionaries.brillonline.com',
    'dictionaries.brillonline.com/montanari',
    'digitalmicrofilm.proquest.com',
    'dl.acm.org',
    'dlcr.alexanderstreet.com',
    'dlib.eastview.com/browse/books/1670',
    'dlib.eastview.com/browse/publication/18406',
    'dlib.eastview.com/browse/udb/350',
    'dlib.eastview.com/sources/publication.jsp?id=21366',
    'dramaonlinelibrary.com',
    'earlyrepublic.press.jhu.edu/index.html',
    'eas-ref.press.jhu.edu/index.html',
    'ebooks.cambridge.org/ebook.jsf?bid=cbo9781139043182',
    'edu.adspender.kantarmediana.com',
    'edu.medici.tv/en',
    'eebo.chadwyck.com/home',
    'eena.alexanderstreet.com',
    'efts.lib.uchicago.edu/efts/voltaire/restricted/search.form.html',
    'elibrary.worldbank.org',
    'esgdirect.msci.com',
    'estc.bl.uk',
    'ets.umdl.umich.edu/m/mec',
    'fconline.foundationcenter.org/ipl.php',
    'fiaf.chadwyck.com/search/initindexsearch.do',
    'fii.chadwyck.com/home',
    'find.galegroup.com/bgmi/infomark.do?page=basicsearch&usergroupname=mlin_m_bostcoll&prodid=bgmi&type=static&version=1.0&source=gale',
    'find.galegroup.com/ecco/start.do?prodid=ecco&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gdl/infomark.do?serquery=locale(en%2c%2c)%3afqe%3d(jx%2cnone%2c23)%22market+share+reporter%22%24&querytype=ph&cpubvalue=0ked&tabid=t045&usergroupname=mlin_m_bostcoll&prodid=gdl&currentposition=1&type=pubissues&version=1.0&source=gale',
    'find.galegroup.com/gdl/infomark.do?serquery=locale(en%2c%2c)%3afqe%3d(jx%2cnone%2c26)%22business+rankings+annual%22%24&querytype=ph&cpubvalue=0htt&tabid=t041&usergroupname=mlin_m_bostcoll&prodid=gdl&currentposition=1&type=pubissues&version=1.0&source=gale',
    'find.galegroup.com/gdl/infomark.do?serquery=locale(en,,):fqe=(jx,none,0)$&querytype=ph&type=pubissues&tabid=t023&cpubvalue=2mox&prodid=gdl&currentposition=1&version=1.0&source=gale&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gdl/infomark.do?serquery=locale(en,,):fqe=(jx,none,0)$&querytype=ph&type=pubissues&tabid=t024&cpubvalue=2moz&prodid=gdl&currentposition=1&version=1.0&source=gale&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gtx/start.do?prodid=itof&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gvrl/start.do?prodid=gvrl.bphandbooks&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gvrl/start.do?prodid=gvrl.encyj&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gvrl/start.do?prodid=gvrl.ncec&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gvrl/start.do?prodid=gvrl.seal&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/gvrl/start.do?prodid=gvrl.wealen&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/iln/start.do?prodid=iln&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/itx/start.do?prodid=aone&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/menu/start?prod=bri&usergroupname=mlin_m_bostcoll',
    'find.galegroup.com/menu/start?usergroupname=mlin_m_bostcoll&prod=ncco',
    'find.galegroup.com/pnch/start.do?prodid=pnch&usergroupname=mlin_m_bostcoll',
    'fmwww.bc.edu/vdc/db/dbdot1a.php',
    'fod.infobase.com/portalplaylists.aspx?wid=95406',
    'galenet.galegroup.com/servlet//ebooks?ste=22&docnum=cx3045399999&q=mlin_m_bostcoll',
    'galenet.galegroup.com/servlet//ebooks?ste=22&docnum=cx3424399999&q=mlin_m_bostcoll',
    'galenet.galegroup.com/servlet/hwrc?locid=mlin_m_bostcoll',
    'gateway.ovid.com/ovidweb.cgi?t=js&mode=ovid&news=n&page=main&d=hapi',
    'gateway.proquest.com/openurl?rft_id=44973&res_dat=xri%3apqm&rft_val_fmt=info%3aofi%2ffmt%3akev%3amtx%3ajournal&url_ver=z39.88-2004&genre=journal&req_dat=xri%3apqil%3aclntid%3d9673',
    'gdc.galegroup.com/gdc/artemis?p=ahsi&u=mlin_m_bostcoll',
    'gdc.galegroup.com/gdc/artemis?p=eapb&u=mlin_m_bostcoll',
    'gdc.galegroup.com/gdc/artemis?p=usdd&u=mlin_m_bostcoll',
    'gerritsen.chadwyck.com',
    'global.factiva.com/en/sess/login.asp?xsid=s00yczpyszymtzymtapm9qmmdaroxmm5dfhy96oyqzlnffbqufbqufbqufbqufbqufbqufbqufbqufbqufbqufbqufb',
    'glossary.agiweb.org/dbtw-wpd/glossary/login.aspx',
    'go.galegroup.com/gdsc/i.do?id=3tqb&v=2.1&u=mlin_m_bostcoll&it=aboutcollections&p=gdsc&sw=w',
    'go.galegroup.com/gdsc/i.do?id=3xnn&v=2.1&u=mlin_m_bostcoll&it=aboutcollections&p=gdsc&sw=w',
    'go.galegroup.com/gdsc/i.do?id=4mtp&v=2.1&u=mlin_m_bostcoll&it=aboutcollections&p=gdsc&sw=w',
    'go.galegroup.com/gdsc/i.do?id=6tjk&v=2.1&u=mlin_m_bostcoll&it=aboutcollections&p=gdsc&sw=w',
    'go.galegroup.com/gdsc/start.do?p=gdsc&u=mlin_m_bostcoll&authcount=1',
    'go.galegroup.com/ps/i.do?v=2.1&u=mlin_m_bostcoll&pg=advancedsearch&it=static&p=gvrl.eb01&sw=w',
    'go.galegroup.com/ps/i.do?v=2.1&u=mlin_m_bostcoll&pg=advancedsearch&it=static&p=itbc&sw=w',
    'go.galegroup.com/ps/start.do?p=itke&u=mlin_m_bostcoll&authcount=1',
    'go.galegroup.com/ps/start.do?p=msic&u=mlin_m_bostcoll&authcount=1',
    'go.galegroup.com/ps/start.do?p=prof&u=mlin_m_bostcoll&authcount=1',
    'go.galegroup.com/ps/start.do?p=stom&u=mlin_m_bostcoll&authcount=1',
    'go.galegroup.com/ps?p=gvrl.0hny&u=mlin_m_bostcoll',
    'goethe.chadwyck.com',
    'gtionline.foundationcenter.org',
    'hanchi.ihp.sinica.edu.tw/ihp/hanji.htm#',
    'hapi.ucla.edu',
    'hbcponline.com',
    'history.paratext.com',
    'hsus.cambridge.org/hsusweb',
    'humanities.uchicago.edu/artfl',
    'ic.galegroup.com/ic/bic1?usergroupname=mlin_m_bostcoll',
    'ic.galegroup.com/ic/gic/?p=gic%3aovic&u=mlin_m_bostcoll',
    'ica.princeton.edu',
    'icsd.fiz-karlsruhe.de/icsd',
    'ieeexplore.ieee.org/xpl/prodlist.jsp?product=&publicationtype=conference&startletter=a',
    'imld.alexanderstreet.com',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=ahsi&id=boston',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=bncn',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=dlbc',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=gld-1',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=hrca',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=lco',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=litrc',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=lt',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=mlfp',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=mmlf',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=mmlp',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=mmlt',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=mome',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=moml',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=ngma',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=noie',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=sata',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=scrb',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=tlsh',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=ttda',
    'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=wmns&id=boston',
    'infoweb.newsbank.com/?db=aspa&s_startsearch=customized',
    'infoweb.newsbank.com/?db=eaix',
    'infoweb.newsbank.com/?db=eana',
    'infoweb.newsbank.com/?db=eanx',
    'infoweb.newsbank.com/?db=eanx&d_collections=eanacn',
    'infoweb.newsbank.com/?db=evan&s_startsearch=customized',
    'infoweb.newsbank.com/?db=fbisx',
    'infoweb.newsbank.com/?db=hsja',
    'infoweb.newsbank.com/?db=seja',
    'infoweb.newsbank.com/?db=serial',
    'infoweb.newsbank.com/?db=shaw&s_startsearch=customized',
    'infoweb.newsbank.com/?db=ssmap',
    'infoweb.newsbank.com/?db=unab',
    'infoweb.newsbank.com/apps/readex/advanced?p=topapart&f=advanced',
    'infoweb.newsbank.com/apps/readex/advanced?p=topcold&f=advanced',
    'infoweb.newsbank.com/apps/readex/advanced?p=topimm&f=advanced',
    'infoweb.newsbank.com/apps/readex/advanced?p=toprace&f=advanced',
    'infoweb.newsbank.com/resources/?p=awnb',
    'infoweb.newsbank.com/resources/search/nb?p=worldnews&t=favorite%3abh%21boston%2bherald%2bhistorical%2band%2bcurrent',
    'infoweb.newsbank.com?db=whnpx&d_collections=whnplan1',
    'isiknowledge.com/biosis',
    'isiknowledge.com/ccc',
    'isiknowledge.com/diidw',
    'isiknowledge.com/drci',
    'isiknowledge.com/inspec',
    'isiknowledge.com/wos',
    'isiknowledge.com/zoorec',
    'iwrp.alexanderstreet.com',
    'jcr.incites.thomsonreuters.com',
    'kafka.chadwyck.com',
    'kll-online.de',
    'knowledge.sagepub.com/view/communicationtheory/sage.xml',
    'laww.alexanderstreet.com',
    'lb.brillonline.nl',
    'leme.library.utoronto.ca/search',
    'libguides.bc.edu/chant',
    'libguides.bc.edu/prowessdx',
    'library.artstor.org/library/welcome.html',
    'library.cqpress.com/congress',
    'library.cqpress.com/cqalmanac',
    'library.cqpress.com/cqresearcher',
    'library.cqpress.com/cqweekly',
    'library.cqpress.com/elections',
    'library.cqpress.com/historicdocuments',
    'library.cqpress.com/phw',
    'library.cqpress.com/pia',
    'library.cqpress.com/scc',
    'library.cqpress.com/statestats',
    'library.nlx.com',
    'link.springer.com',
    'link.springer.com/referencework/10.1007/978-1-349-58802-2',
    'literature.proquest.com/?accountid=9673',
    'litguide.press.jhu.edu',
    'luther.chadwyck.com',
    'materials.springer.com',
    'mathscinet.ams.org/mathscinet',
    'metasearch.opinionarchives.com/advanced.html',
    'methods.sagepub.com/cases',
    'methods.sagepub.com/datasets',
    'methods.sagepub.com/search',
    'metopera.org/season/on-demand/?',
    'microform.digital/boa/collections/11/america-in-records-from-colonial-missionaries-1635-1928',
    'microform.digital/boa/search?facettopic=tradeinpeople',
    'microform.digital/boa/search?facettopic=tww',
    'mio.asminternational.org/apd',
    'moneyline.cq.com/pml/home.do',
    'monthlycatalog.chadwyck.com',
    'moto.alexanderstreet.com',
    'mrw.interscience.wiley.com/emrw/9780470682531/patai/toc',
    'muse.jhu.edu',
    'myinsight.globalinsight.com/servlet/cats?imask=1&pagecontent=home',
    'na.capitaliq.com/ip/bosto',
    'nano.nature.com',
    'newfirstsearch.oclc.org/autho=100105433&dbname=worldcat&done=referer&fsip',
    'news.siteintelgroup.com',
    'next.srds.com',
    'ntrl.ntis.gov',
    'nwld.alexanderstreet.com',
    'online.epocrates.com/noframe',
    'online.goinglobal.com',
    'online.ibfd.org/kbase',
    'online.infobaselearning.com/direct.aspx?aid=95406&pid=we00',
    'online.liverpooluniversitypress.co.uk/series/tthe',
    'onlinelibrary.wiley.com',
    'onlinelibrary.wiley.com/book/10.1002/0470848944',
    'onlinelibrary.wiley.com/book/10.1002/047120918x',
    'onlinelibrary.wiley.com/book/10.1002/0471264180',
    'onlinelibrary.wiley.com/book/10.1002/9780470671153/toc',
    'onlinelibrary.wiley.com/book/10.1002/9781118829059/toc',
    'onlinelibrary.wiley.com/book/10.1107/97809553602060000001',
    'onlinelibrary.wiley.com/mrw/advanced/search?doi=10.1002/047001590x',
    'onlinelibrary.wiley.com/mrw/advanced/search?doi=10.1002/0471238961',
    'oversea.cnki.net',
    'ovidsp.ovid.com/ovidweb.cgi?t=js&mode=ovid&page=main&news=n&d=pmoz',
    'ovidsp.ovid.com/ovidweb.cgi?t=js&news=n&csc=y&page=main&d=jbi',
    'papers.nber.org/papers',
    'parkerweb.stanford.edu',
    'parlipapers.proquest.com/profiles/hcpp/search/advanced/hcppadvanced?accountid=9673',
    'phaseonline.ceramics.org',
    'philpapers.org',
    'pld.chadwyck.com',
    'pm.nlx.com',
    'poll.orspub.com/poll/lpext.dll?f=templates&fn=main-h.htm',
    'primarysources.brillonline.com/advanced-search?collection=latin-american-anarchist-periodicals',
    'primarysources.brillonline.com/browse/moses-maimonides-unparalleled-editions',
    'primarysources.brillonline.com/browse/us-intelligence-on-the-middle-east',
    'projecteuclid.org',
    'proquestcombo.safaribooksonline.com',
    'pubs.acs.org/about.html',
    'pubs.rsc.org',
    'purl.org/reltech/migne',
    'quod.lib.umich.edu/a/acls/browse.html',
    'readyreference.data-planet.com',
    'reference.paratext.com',
    'referenceworks.brillonline.com/browse/brockelmann',
    'referenceworks.brillonline.com/browse/christian-muslim-relations-i',
    'referenceworks.brillonline.com/browse/christian-muslim-relations-ii',
    'referenceworks.brillonline.com/browse/encyclopaedia-islamica',
    'referenceworks.brillonline.com/browse/encyclopaedia-of-islam-2',
    'referenceworks.brillonline.com/browse/encyclopaedia-of-islam-3',
    'referenceworks.brillonline.com/browse/encyclopaedia-of-judaism',
    'referenceworks.brillonline.com/browse/encyclopaedia-of-the-neo-latin-world',
    'referenceworks.brillonline.com/browse/encyclopaedia-of-the-quran',
    'referenceworks.brillonline.com/browse/encyclopedia-of-arabic-language-and-linguistics',
    'referenceworks.brillonline.com/browse/encyclopedia-of-early-modern-history-online',
    'referenceworks.brillonline.com/browse/encyclopedia-of-jews-in-the-islamic-world',
    'referenceworks.brillonline.com/browse/encyclopedia-of-law-and-religion',
    'referenceworks.brillonline.com/browse/encyclopedia-of-women-and-islamic-cultures',
    'referenceworks.brillonline.com/browse/gregorii-nysseni-opera',
    'referenceworks.brillonline.com/browse/jesuit-historiography-online',
    'referenceworks.brillonline.com/browse/religion-past-and-present',
    'referenceworks.brillonline.com/cluster/new%20pauly%20online',
    'religion.oxfordre.com',
    'research.valueline.com/secure/research',
    'ropercenter.cornell.edu',
    'rotunda.upress.virginia.edu/arnold',
    'rotunda.upress.virginia.edu/crossetti',
    'rotunda.upress.virginia.edu/dmde',
    'rotunda.upress.virginia.edu/edc',
    'rotunda.upress.virginia.edu/founders/adms.html',
    'rotunda.upress.virginia.edu/founders/arhn.html',
    'rotunda.upress.virginia.edu/founders/fgea.html',
    'rotunda.upress.virginia.edu/founders/gewn.html',
    'rotunda.upress.virginia.edu/founders/jsmn.html',
    'rotunda.upress.virginia.edu/founders/rncn',
    'rotunda.upress.virginia.edu/founders/tsjn.html',
    'rotunda.upress.virginia.edu/pinckneyhorry',
    'schiller.chadwyck.com',
    'school.eb.com/levels',
    'search.alexanderstreet.com/afso',
    'search.alexanderstreet.com/ativ',
    'search.alexanderstreet.com/blsv',
    'search.alexanderstreet.com/bord',
    'search.alexanderstreet.com/counseling-therapy',
    'search.alexanderstreet.com/daiv',
    'search.alexanderstreet.com/ediv',
    'search.alexanderstreet.com/glnd',
    'search.alexanderstreet.com/huri',
    'search.alexanderstreet.com/meet',
    'search.alexanderstreet.com/music-performing-arts',
    'search.alexanderstreet.com/opiv',
    'search.alexanderstreet.com/wasg',
    'search.alexanderstreet.com/wasi',
    'search.alexanderstreet.com/wnrv',
    'search.eb.com',
    'search.ebscohost.com/login.asp?profile=ahw',
    'search.ebscohost.com/login.asp?profile=religion&defaultdb=lsdar',
    'search.ebscohost.com/login.asp?profile=web&defaultdb=bwh',
    'search.ebscohost.com/login.aspx?authtype=cookie,ip,uid&profile=advance&defaultdb=ahl',
    'search.ebscohost.com/login.aspx?authtype=cookie,ip,uid&profile=advance&defaultdb=hia',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&custid=s4194587&profile=ehost&defaultdb=apd,ane,apm,afp,alm,bqp,bgp,tpp,bsj,bsi,cap,clp,csp,cos,cpp,cps,cep,dap,amr,fcf,flp,gic,hss,lpn,msp,mle,mcp,mpe,pfa,paw,pbe,eup,rpw,rpf,scp,sla,sdd,ssp,tpa,bsk,wpe,wpd,wpn,ald,alt,amp,app,acw,aml,asp&groupid=main&scope=site',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=advance&defaultdb=fqh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=advance&defaultdb=oah',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=advance&defaultdb=vah',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=bsi',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=cin&defaultdb=jlh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=33h',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=8gh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=air',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=ant',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=apn&defaultdb=alr',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=asu',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=bas',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=brd,brr',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=cax&defaultdb=ufh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=egi',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=eir',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=fxh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=fyh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=gnh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=h6a',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=h7h&defaultdb=h8h',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=hjh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=hsr',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=ich',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=ijh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=jph',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=llf',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=loh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=lpb&defaultdb=lpr',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=lxh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=pix',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=qth',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=rgr',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=rih',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=rvh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=sbh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=sph',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=ssx',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=swh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=erc&defaultdb=ehh',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=novelistk8',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=religion&defaultdb=pif',
    'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=srchall&defaultdb=bpr',
    'search.epnet.com/login.asp?profile=web&defaultdb=mah',
    'search.itergateway.org/iter/search.jsp',
    'search.marquiswhoswho.com',
    'search.proquest.com/advanced?accountid=9673&selectids=1005629,1007871',
    'search.proquest.com/advanced?accountid=9673&selectids=1005629,1007871,1007272,1006744,1006151,1007155,1007154,1006359,1005877,1006360',
    'search.proquest.com/advanced?accountid=9673&selectids=1007603,1007604',
    'search.proquest.com/agricenvironm/advanced/science/fromdatabaseslayer?accountid=9673',
    'search.proquest.com/altpresswatch/advanced?accountid=9673',
    'search.proquest.com/americanperiodicals/advanced?accountid=9673',
    'search.proquest.com/artbibliographies/advanced?',
    'search.proquest.com/artshumanities/advanced?accountid=9673',
    'search.proquest.com/asfa/science/fromdatabaseslayer?accountid=9673',
    'search.proquest.com/assia/advanced?',
    'search.proquest.com/avery/advanced?',
    'search.proquest.com/bhi/advanced?',
    'search.proquest.com/biologicalsciences/advanced?accountid=9673',
    'search.proquest.com/ble/advanced?accountid=9673',
    'search.proquest.com/britishnursingindex/advanced?accountid=9673',
    'search.proquest.com/britishperiodicals/advanced?accountid=9673',
    'search.proquest.com/criminaljusticeperiodicals/advanced?accountid=9673',
    'search.proquest.com/daai/advanced?',
    'search.proquest.com/dnsa/advanced?accountid=9673',
    'search.proquest.com/earthatmosphericaquatic/advanced/science/fromdatabaseslayer?accountid=9673',
    'search.proquest.com/econlit/advanced?',
    'search.proquest.com/education/advanced?accountid=9673',
    'search.proquest.com/eric/advanced?accountid=9673',
    'search.proquest.com/ethnicnewswatch/advanced?accountid=9673',
    'search.proquest.com/genderwatch/advanced?accountid=9673',
    'search.proquest.com/globalnews/embedded/sv46nqy98qsi5agf?_ga=2.24403762.927166705.1519997644-1600456357.1519997644',
    'search.proquest.com/harpersbazaar/advanced/arts/fromdatabaseslayer?accountid=9673',
    'search.proquest.com/histvault?q=395:000579',
    'search.proquest.com/histvault?q=48990:001738',
    'search.proquest.com/histvault?q=91549:001475',
    'search.proquest.com/hnpatlantaconstitution2/advanced?accountid=9673',
    'search.proquest.com/hnpchicagodefender/advanced?accountid=9673',
    'search.proquest.com/hnpchicagotribune/advanced?accountid=9673',
    'search.proquest.com/hnpchristiansciencemonitor/advanced?accountid=9673',
    'search.proquest.com/hnpglobeandmail?accountid=9673',
    'search.proquest.com/hnpguardianobserver/advanced/?accountid=9673',
    'search.proquest.com/hnpirishtimes/advanced?accountid=9673',
    'search.proquest.com/hnplatimes?accountid=9673',
    'search.proquest.com/hnpnewamsterdamnews/advanced?accountid=9673',
    'search.proquest.com/hnpnewyorktimes/advanced?accountid=9673',
    'search.proquest.com/hnpscotsman/advanced?accountid=9673',
    'search.proquest.com/hnptimesofindia/advanced?accountid=9673',
    'search.proquest.com/hnpwallstreetjournal/advanced?accountid=9673',
    'search.proquest.com/hnpwashingtonpost/advanced?accountid=9673',
    'search.proquest.com/iba/advanced?accountid=9673',
    'search.proquest.com/ibss/advanced?accountid=9673',
    'search.proquest.com/iimp/advanced?accountid=9673',
    'search.proquest.com/iipa/advanced?accountid=9673',
    'search.proquest.com/llba/advanced?accountid=9673',
    'search.proquest.com/mga/science/fromdatabaseslayer',
    'search.proquest.com/mlaib/advanced?',
    'search.proquest.com/naturalscience/advanced/science/fromdatabaseslayer?accountid=9673',
    'search.proquest.com/oceanic/advanced?',
    'search.proquest.com/pais/advanced?',
    'search.proquest.com/pao/advanced?accountid=9673',
    'search.proquest.com/physicaleducation/advanced?',
    'search.proquest.com/pilots/advanced?',
    'search.proquest.com/pio/advanced?accountid=9673',
    'search.proquest.com/policyfile/advanced?accountid=9673',
    'search.proquest.com/politicalscience/advanced?accountid=9673',
    'search.proquest.com/pqdtglobal/advanced?accountid=9673',
    'search.proquest.com/psycarticles/advanced?accountid=9673',
    'search.proquest.com/psycbooks/advanced?',
    'search.proquest.com/psycinfo/advanced?',
    'search.proquest.com/psyctests/advanced?accountid=9673',
    'search.proquest.com/socabs/advanced?',
    'search.proquest.com/socialservices/advanced?',
    'search.proquest.com/sociology/advanced?accountid=9673',
    'search.proquest.com/socscijournals/advanced?accountid=9673',
    'search.proquest.com/toxline/advanced?accountid=9673',
    'search.proquest.com/usmajordailies/embedded/sv46nqy98qsi5agf?_ga=2.237868168.927166705.1519997644-1600456357.1519997644',
    'search.proquest.com/vogue/advanced?accountid=9673',
    'search.proquest.com/wma?accountid=9673',
    'search.proquest.com/wpsa/advanced?accountid=9673',
    'search.proquest.com/wwd/advanced?accountid=9673',
    'shmu.alexanderstreet.com',
    'sk.sagepub.com/cases',
    'sk.sagepub.com/reference/humanrelationships',
    'sk.sagepub.com/reference/journalism',
    'sk.sagepub.com/search',
    'sk.sagepub.com/search/savedsearch/280',
    'sk.sagepub.com/video/business-management',
    'solomon.dkbl.alexanderstreet.com',
    'solomon.tcpt.alexanderstreet.com',
    'stat.unido.org',
    'statabs.proquest.com/sa/index.html?accountid=9673',
    'statistical.proquest.com/statisticalinsight/search/advanced/si?accountid=9673&groupid=107819',
    'statisticaldatasets.data-planet.com/dataplanet',
    'statisticaldatasets.data-planet.com/dataplanet?open=bop',
    'statisticaldatasets.data-planet.com/dataplanet?open=dot',
    'statisticaldatasets.data-planet.com/dataplanet?open=ifs',
    'stephanus.tlg.uci.edu/iris/inst/csearch.jsp',
    'swrp.alexanderstreet.com',
    'syrano.demarque.com/access/connext.php?idrelpc=372596',
    'tapor.library.utoronto.ca/doe',
    'tapor.library.utoronto.ca/doecorpus',
    'teso.chadwyck.com/home/home',
    'totalaccess.emarketer.com/extranet.aspx?bostoncollege',
    'trials.proquest.com/trials/trialsummary.action?view=subject&trialbean.token=csrde8xxjivge9gv69un',
    'tvnews.vanderbilt.edu',
    'universitypublishingonline.org/cambridge/benjonson',
    'universitypublishingonline.org/cambridge/companions',
    'universitypublishingonline.org/cambridge/companions/browse_by_collection.jsf?collectiontitle=companions+to+music&subproductgroupcode=mus&tsort=title+closed&asort=author+default_list&ysort=year+default_list',
    'universitypublishingonline.org/cambridge/histories',
    'universitypublishingonline.org/cambridge/histories/subject_title_list.jsf?subjectcode=09&heading=music&tsort=title+closed&asort=author+default_list&ysort=year+default_list',
    'voices.revealdigital.com',
    'web.lexis-nexis.com/stcapuniv',
    'webcsd.ccdc.cam.ac.uk',
    'webofknowledge.com/?destapp=wos&editions=ahci',
    'webofknowledge.com/?destapp=wos&editions=isshp',
    'webofknowledge.com/?destapp=wos&editions=istp',
    'webofknowledge.com/?destapp=wos&editions=sci',
    'webofknowledge.com/?destapp=wos&editions=ssci',
    'wellesley.chadwyck.com',
    'wnc.eastview.com',
    'worldchristiandatabase.org/wcd',
    'wrds-web.wharton.upenn.edu/wrds/query_forms/navigation.cfm?navid=336',
    'www.18thcjournals.amdigital.co.uk',
    'www.aapredbook.org',
    'www.academic.marketresearch.com',
    'www.accessible-archives.com/collections/african-american-newspapers',
    'www.accessible.com/accessible/brand?mpthe+lily=on&mpnational+citizen+and+ballot+box=on&mpthe+revolution=on',
    'www.accessscience.com',
    'www.airitilibrary.com/search/aljnlbrowse',
    'www.alacra.com/cgi-bin/eiusite.exe?app=eiusite&msg=execcontent&topic=defaultscreen/exlibris/metalib/m4_1/product=17',
    'www.anb.org/articles/index.html',
    'www.archivesdirect.amdigital.co.uk/cp_middleeast',
    'www.archivesdirect.amdigital.co.uk/fo_india',
    'www.archivesdirect.amdigital.co.uk/fo_japan',
    'www.archivesdirect.amdigital.co.uk/fo_middleeast',
    'www.askart.com',
    'www.balkaninsight.com/en/page/all-balkans-home',
    'www.bc.edu/content/dam/files/schools/law_sites/library/html/bna/alpha.html',
    'www.bdsl-online.de',
    'www.beckettarchive.org',
    'www.bibliorossica.com',
    'www.biocyc.org',
    'www.blackwellreference.com',
    'www.booksinprint.com',
    'www.brepolis.net',
    'www.brepolsonline.net',
    'www.bridgemaneducation.com',
    'www.britishonlinearchives.co.uk/collection.php?cid=9781851171248&sid=&keywords=',
    'www.britishonlinearchives.co.uk/collection.php?cid=9781851171385&pid=&did=&cat=&sid=boabbc&date_option=equal',
    'www.buddhism-dict.net/ddb',
    'www.cabells.com',
    'www.cepr.org/content/discussion-papers',
    'www.chicagomanualofstyle.org/home.html',
    'www.china.amdigital.co.uk',
    'www.churchillarchive.com',
    'www.ciaonet.org',
    'www.cios.org',
    'www.classiques-garnier.com/numerique-bases/grammaires',
    'www.classiques-garnier.com/numerique-bases/index.php?module=app&action=framemain&colname=colbasile',
    'www.clcd.com/#/advancedsearch',
    'www.cochranelibrary.com',
    'www.collegesource.org',
    'www.colonialamerica.amdigital.co.uk',
    'www.communicationencyclopedia.com',
    'www.computingreviews.com',
    'www.conference-board.org/ea/index.cfm',
    'www.corporateaffiliations.com/executable/cn_quicksearch.asp',
    'www.crl.edu/content.asp?l1=5&l2=23&l3=41',
    'www.daredictionary.com',
    'www.degruyter.com/databasecontent?dbid=ebr&dbsource=%2fdb%2febr',
    'www.digitaltheatreplus.com',
    'www.digizeitschriften.de',
    'www.duxiu.com/login.jsp',
    'www.e-enlightenment.com',
    'www.eastview.com/online/sikudownload',
    'www.eduref.org',
    'www.embase.com',
    'www.emeraldinsight.com/search/advanced',
    'www.emis.com/php/dashboard/index',
    'www.empire.amdigital.co.uk',
    'www.englishhistoricaldocuments.com',
    'www.ethnologue.com/browse/names',
    'www.europeanamericana.com',
    'www.foreignaffairs.com',
    'www.foreignlawguide.com/ip',
    'www.geoscienceworld.org',
    'www.globalcommodities.amdigital.co.uk',
    'www.guidestar.org',
    'www.heinonline.org/hol/index?collection=iflp',
    'www.heinonline.org/hol/welcome',
    'www.hstalks.com/business',
    'www.icpsr.umich.edu',
    'www.ieee.org/ieeexplore',
    'www.ingentaconnect.com',
    'www.ingentaconnect.com/content/mmso',
    'www.iop.org/ej/search',
    'www.irishbooksearch.com/index.php?signin=y',
    'www.issgovernance.com/solutions/iss-analytics/iss-incentive-lab',
    'www.jstor.org/action/showadvancedsearch',
    'www.jstor.org/action/showjournals?browsetype=collectioninfopage&selectcollection=ireland',
    'www.jstor.org/sustainability',
    'www.kluwerlawonline.com/toc.php?area=looseleafs&mode=bypub&level=4&values=looseleafs~~iel+labour+law',
    'www.knovel.com/knovel2/toc.jsp?bookid=761&verticalid=0',
    'www.knovel.com/knovel2/toc.jsp?spaceid=10093&bookid=735',
    'www.latinnews.com',
    'www.lib.uchicago.edu/efts/artfl/databases/bibliopolis/cli',
    'www.lib.uchicago.edu/efts/artfl/databases/bibliopolis/cphil',
    'www.lib.uchicago.edu/efts/artfl/databases/bibliopolis/rom',
    'www.lib.uchicago.edu/efts/artfl/databases/bibliopolis/rrn',
    'www.lib.uchicago.edu/efts/artfl/databases/bibliopolis/tgs',
    'www.lib.uchicago.edu/efts/artfl/databases/prov',
    'www.lib.uchicago.edu/efts/artfl/projects/fww',
    'www.llmcdigital.org',
    'www.lobbyists.info',
    'www.loebclassics.com',
    'www.lyellcollection.org',
    'www.massobservation.amdigital.co.uk',
    'www.medievalfamilylife.amdigital.co.uk',
    'www.medievaltravel.amdigital.co.uk/search/advanced-search.aspx',
    'www.mergentarchives.com',
    'www.mergentintellect.com',
    'www.mergentonline.com/reportssearch.php?pagetype=industrysearch&',
    'www.morganclaypool.com/search/advanced',
    'www.nature.com/news',
    'www.nature.com/scientificamerican/archive/index.html',
    'www.nature.com/siteindex/index.html',
    'www.ncbi.nlm.nih.gov/entrez/query.fcgi?otool=maubnulib',
    'www.nexisuni.com',
    'www.nursinglibrary.org/vhl',
    'www.nybooks.com/archives',
    'www.oecd-ilibrary.org',
    'www.oecd-ilibrary.org/statistics',
    'www.oed.com',
    'www.osapublishing.org/search.cfm',
    'www.oxfordaasc.com',
    'www.oxfordartonline.com',
    'www.oxfordartonline.com/benezit',
    'www.oxfordbiblicalstudies.com',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780190280024',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195389661',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195390155',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195393361',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195393521',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195396577',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195396584',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195396607',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780195399318',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199730414',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199743292',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199756384',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199756797',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199756841',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199757824',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199766567',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199766581',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199772810',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199791231',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199791279',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199791286',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199799558',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199827251',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199828340',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199830060',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199840731',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199846719',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199846733',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199846740',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199874002',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199913701',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199920082',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199920105',
    'www.oxfordbibliographies.com/browse?module_0=obo-9780199941728',
    'www.oxfordbibliographies.com/obo/page/political-science',
    'www.oxfordbibliographies.com/obo/page/renaissance-and-reformation',
    'www.oxfordbibliographies.com/obo/page/social-work',
    'www.oxfordbibliographies.com/view/document/obo-9780199791231/obo-9780199791231-0024.xml',
    'www.oxforddictionaries.com',
    'www.oxforddnb.com',
    'www.oxfordhandbooks.com/page/psychology',
    'www.oxfordhandbooks.com/page/religion',
    'www.oxfordislamicstudies.com',
    'www.oxfordmusiconline.com',
    'www.oxfordreference.com',
    'www.oxfordreference.com.proxy.bc.edu/view/10.1093/acref/9780191842665.001.0001/acref-9780191842665',
    'www.oxfordreference.com/view/10.1093/acref/9780195125955.001.0001/acref-9780195125955',
    'www.oxfordreference.com/view/10.1093/acref/9780195169218.001.0001/acref-9780195169218',
    'www.oxfordreference.com/view/10.1093/acref/9780195176322.001.0001/acref-9780195176322',
    'www.oxfordreference.com/view/10.1093/acref/9780195189483.001.0001/acref-9780195189483',
    'www.oxfordreference.com/view/10.1093/acref/9780195313918.001.0001/acref-9780195313918#showmorecontent',
    'www.oxfordreference.com/views/book_search.html?book=t249',
    'www.oxfordscholarlyeditions.com',
    'www.palgraveconnect.com/pc/doifinder/10.1057/9781137305688',
    'www.pep-web.org',
    'www.perditamanuscripts.amdigital.co.uk',
    'www.perseus.tufts.edu',
    'www.psychiatryonline.com',
    'www.reaxys.com',
    'www.referenceusa.com',
    'www.rep.routledge.com/search?showadvancedsearch',
    'www.responsa.co.il/default.aspx',
    'www.rkma.com/bc/oneill',
    'www.rocksbackpages.com',
    'www.romanticism.amdigital.co.uk',
    'www.ruhr-uni-bochum.de/philosophy/projects/abbreviationes/servers.html',
    'www.sciencedirect.com/science/referenceworks/9780080430768',
    'www.sciencedirect.com/science/referenceworks/9780080931395',
    'www.sciencedirect.com/science/referenceworks/9780080983004',
    'www.sciencedirect.com/science/referenceworks/9780123694010',
    'www.sciencedirect.com/science/referenceworks/9780123708700',
    'www.sciencedirect.com/science/referenceworks/9780123744739',
    'www.sciencedirect.com/science/referenceworks/9780444527486',
    'www.sciencedirect.com/science/referenceworks/9780444538031',
    'www.sciencedirect.com/science?_ob=journallisturl&_type=all&_auth=y&_btn=y&_acct=c000026018&_version=1&_urlversion=0&_userid=521319&md5=99ce3a582d5a4b4a50ecec8bb1fb420c',
    'www.scitation.org',
    'www.scopus.com',
    'www.snl.com/interactivex',
    'www.socialexplorer.com/ezproxy',
    'www.socialismonfilm.amdigital.co.uk',
    'www.spiedigitallibrary.org',
    'www.springerlink.com/content/978-1-4020-9211-4',
    'www.springerlink.com/content/m46752/?sortorder=asc&v=condensed',
    'www.stalindigitalarchive.com',
    'www.statcan.gc.ca/start-debut-eng.html',
    'www.statehousenews.com/public/default.htm',
    'www.statindex.org/cis/psqlquery',
    'www.statista.com',
    'www.tandfonline.com/action/dosearch?allfield=encyclopedia+of+natural+resources&content=title&target=titlesearch',
    'www.teachingbooks.net',
    'www.tol.org',
    'www.trackingterrorism.org',
    'www.tradelawguide.com',
    'www.ulrichsweb.com/ulrichsweb',
    'www.victorianperiodicals.com',
    'www.wisertrade.org/ftlib10web/ftbegin',
    'www.wkcheetah.com',
    'www.wkcheetah.com/#/home/bluechip',
    'www.worldreligiondatabase.org/wrd_home.asp',
    'www.worldshakesbib.org/index.html',
    'www.worldtradelaw.net',
    'www.wwp.northeastern.edu/wwo',
    'www.zbmath.org',
    'www3.interscience.wiley.com/cgi-bin/mrwhome/104554785/home',
    'ybio.brillonline.com'
]
db_names_set = set(db_names)
print(len(db_names_set))

735


In [164]:
# parse guides
guides_parsed_count = 0
pages_parsed_count = 0
boxes_parsed_count = 0
assets_parsed_count = 0
link_assets_parsed_count = 0

proxied_asset_count = 0
proxied_assets_list = []

suspicious_asset_count = 0
suspicious_asset_list = []

# parse guides
for guide in root.findall('./guides/guide'):
    guide_id = guide.findtext('id')
    guide_name = guide.findtext('name')
    guide_url = guide.findtext('url')
    guide_owner = guide.findtext('owner/id')
    
    # parse pages in a guide
    local_pages_count = 0
    pages_dict = {}
    for page in guide.findall('./pages/page'):
        page_id = page.findtext('id')
        page_name = page.findtext('name')
        page_url = page.findtext('url')
        
        # parse boxes in a page
        local_boxes_count = 0
        boxes_dict = {}
        for box in page.findall('./boxes/box'):
            box_id = box.findtext('id')
            box_name = box.findtext('name')
            
            # construct box_url
            box_url = "%s#s-lg-box-%s" % (page_url, box_id)
            
            # parse assets in a box
            local_assets_count = 0
            assets_dict = {}
            for asset in box.findall('./assets/asset'):
                asset_id = asset.findtext('id')
                asset_name = asset.findtext('name')
                asset_type = asset.findtext('type')
                
                # check if this asset is of type "Link"
                if asset_type == "Link":
                    asset_url_raw = asset.findtext('url')
                    
                    if not asset_url_raw:
                        # TODO log error here
                        continue
                    
                    # shift to lowercase 
                    asset_url_lower = asset_url_raw.lower()
                    
                    # strip out protocol (http:// | https://)
                    asset_url_clean = re.sub(remove_protocol_re, "", asset_url_lower)
                    
                    # strip out trailing forward slash
                    asset_url_clean = asset_url_clean.rstrip('/')
                    
                    # first, check if this has the proxy prefix
                    has_proxy = proxy_re.match(asset_url_clean)
                    if has_proxy:
                        asset_url = re.sub(proxy_re, '', asset_url_clean)
                        
                        # add to proxied_assets_list
                        proxied_assets_list.append(
                            {
                                "guide_id": guide_id,
                                "guide_owner": accounts_dict[guide_owner],
                                "guide_name": guide_name,
                                # "guide_url": guide_url,
                                # "page_id": page_id,
                                "page_name": page_name,
                                # "page_url": page_url,
                                # "box_id": box_id,
                                "box_name": box_name,
                                "box_url": box_url,
                                # "asset_id": asset_id,
                                "asset_name": asset_name,
                                "asset_url_raw": asset_url_raw,
                                "matches_on_known_db_name": asset_url,
                                "reason": "uses proxy prefix"
                            }
                        )
                        proxied_asset_count += 1
                    else:
                        asset_url = asset_url_clean
                        
                    
                    # next, check if this url matches the full database url
                    # if not, then check to see if the asset_url matches a db domain name
                    if asset_url in db_names_set:
                        suspicious_asset_list.append(
                            {
                                "guide_id": guide_id,
                                "guide_owner": accounts_dict[guide_owner],
                                "guide_name": guide_name,
                                # "guide_url": guide_url,
                                # "page_id": page_id,
                                "page_name": page_name,
                                # "page_url": page_url,
                                # "box_id": box_id,
                                "box_name": box_name,
                                "box_url": box_url,
                                # "asset_id": asset_id,
                                "asset_name": asset_name,
                                "asset_url_raw": asset_url_raw,
                                "matches_on_known_db_name": asset_url,
                                "reason": "matches full database url"
                            }
                        )
                        suspicious_asset_count += 1
                    else: 
                        # extract out the domain name from the asset_url variable
                        get_dn = domain_name_re.search(asset_url)
                        if get_dn:
                            matched_dn = get_dn.group("dn")
                            if matched_dn:
                                asset_url_dn = matched_dn
                            else:
                                # TODO log an error here
                                continue

                        # match asset_url_dn to known database dn's
                        if asset_url_dn in db_domain_names_set:
                            # add to suspicious_asset_list
                            suspicious_asset_list.append(
                                {
                                    "guide_id": guide_id,
                                    "guide_owner": accounts_dict[guide_owner],
                                    "guide_name": guide_name,
                                    # "guide_url": guide_url,
                                    # "page_id": page_id,
                                    "page_name": page_name,
                                    # "page_url": page_url,
                                    # "box_id": box_id,
                                    "box_name": box_name,
                                    "box_url": box_url,
                                    # "asset_id": asset_id,
                                    "asset_name": asset_name,
                                    "asset_url_raw": asset_url_raw,
                                    "matches_on_known_db_name": asset_url_dn,
                                    "reason": "matches a database domain name"
                                }
                            )
                            suspicious_asset_count += 1

                    link_assets_parsed_count += 1
                    # END check if this asset is of type "Link"
                
                local_assets_count += 1
                assets_parsed_count += 1
                # END parse assets in a box
            
            local_boxes_count += 1
            boxes_parsed_count += 1
            # END parse boxes in a page
            
        local_pages_count += 1
        pages_parsed_count += 1
        # END parse pages in a guide
        
    guides_parsed_count += 1
    # END parse guides

In [165]:
print("guides_parsed_count: %s" % guides_parsed_count)
print("pages_parsed_count: %s" % pages_parsed_count)
print("boxes_parsed_count: %s" % boxes_parsed_count)
print("assets_parsed_count: %s" % assets_parsed_count)
print("link_assets_parsed_count: %s" % link_assets_parsed_count)
print("proxied_asset_count: %s" % proxied_asset_count)
print("suspicious_asset_count: %s" % suspicious_asset_count)

guides_parsed_count: 813
pages_parsed_count: 4648
boxes_parsed_count: 13346
assets_parsed_count: 29366
link_assets_parsed_count: 14071
proxied_asset_count: 1013
suspicious_asset_count: 2073


In [166]:
pprint.pprint(proxied_assets_list)

[{'asset_name': 'National Bureau of Economic Research',
  'asset_url_raw': 'https://proxy.bc.edu/login?url=http://www.nber.org/wwp.html',
  'asset_url_to_match': 'www.nber.org/wwp.html',
  'box_name': 'Directories in Print',
  'box_url': 'https://libguides.bc.edu/c.php?g=2346&p=7505#s-lg-box-16432',
  'guide_id': '2346',
  'guide_name': "Jesssse's Test LibGuide",
  'guide_owner': 'Jesse Martinez',
  'page_name': 'Directories/Guides',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'National Bureau of Economic Research',
  'asset_url_raw': 'https://proxy.bc.edu/login?url=http://www.nber.org/wwp.html',
  'asset_url_to_match': 'www.nber.org/wwp.html',
  'box_name': 'Statistics From and About the U.S. Government',
  'box_url': 'https://libguides.bc.edu/jesse/stats#s-lg-box-16436',
  'guide_id': '2346',
  'guide_name': "Jesssse's Test LibGuide",
  'guide_owner': 'Jesse Martinez',
  'page_name': 'Statistics',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Science',
  'asset_url_raw': '

  'box_url': 'https://libguides.bc.edu/c.php?g=43979&p=279340#s-lg-box-833640',
  'guide_id': '43979',
  'guide_name': 'Marketing & Advertising',
  'guide_owner': 'Sonia Ensins',
  'page_name': 'Demographics & Target Markets',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Statistical Abstract of the United States.',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=https://search.proquest.com/statistical?landingpage=statab',
  'asset_url_to_match': 'search.proquest.com/statistical?landingpage=statab',
  'box_name': 'Demographics & Target Markets',
  'box_url': 'https://libguides.bc.edu/c.php?g=43979&p=279340#s-lg-box-833640',
  'guide_id': '43979',
  'guide_name': 'Marketing & Advertising',
  'guide_owner': 'Sonia Ensins',
  'page_name': 'Demographics & Target Markets',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Vanderbilt Television News Archive',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://tvnews.vanderbilt.edu/',
  'asset_url_to_match': 'tvnews.vanderbilt.e

  'reason': 'uses proxy prefix'},
 {'asset_name': 'Irish Newspaper Archives',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://irishnewspaperarchives.com/iplogin.aspx',
  'asset_url_to_match': 'irishnewspaperarchives.com/iplogin.aspx',
  'box_name': 'Quick Database Links',
  'box_url': 'https://libguides.bc.edu/c.php?g=44005&p=279510#s-lg-box-834271',
  'guide_id': '44005',
  'guide_name': 'Irish Studies General Reference',
  'guide_owner': 'Kathy Williams',
  'page_name': 'Home',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'JSTOR Ireland',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.jstor.org/action/showAdvancedSearch',
  'asset_url_to_match': 'www.jstor.org/action/showadvancedsearch',
  'box_name': 'Quick Database Links',
  'box_url': 'https://libguides.bc.edu/c.php?g=44005&p=279510#s-lg-box-834271',
  'guide_id': '44005',
  'guide_name': 'Irish Studies General Reference',
  'guide_owner': 'Kathy Williams',
  'page_name': 'Home',
  'reason': 'uses proxy

  'box_name': 'Biography Databases',
  'box_url': 'https://libguides.bc.edu/theaterstudies/biographies#s-lg-box-837783',
  'guide_id': '44029',
  'guide_name': 'Theater Studies',
  'guide_owner': 'Nina Bogdanovsky',
  'page_name': 'Biographical Sources',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'A Dictionary of Opera Characters',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.oxfordreference.com/views/BOOK_SEARCH.html?book=t78',
  'asset_url_to_match': 'www.oxfordreference.com/views/book_search.html?book=t78',
  'box_name': 'Biography Databases',
  'box_url': 'https://libguides.bc.edu/theaterstudies/biographies#s-lg-box-837783',
  'guide_id': '44029',
  'guide_name': 'Theater Studies',
  'guide_owner': 'Nina Bogdanovsky',
  'page_name': 'Biographical Sources',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Biography and Genealogy Master Index',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://proxy.bc.edu/login?url=http://proxy.bc.edu/login?url=http:/

  'reason': 'uses proxy prefix'},
 {'asset_name': 'Literature Criticism Online',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=LCO',
  'asset_url_to_match': 'infotrac.galegroup.com/itweb/mlin_m_bostcoll?db=lco',
  'box_name': 'Peer-Reviewed Articles: Literature Databases',
  'box_url': 'https://libguides.bc.edu/aframlit/articles#s-lg-box-1716898',
  'guide_id': '44042',
  'guide_name': 'African American Literature',
  'guide_owner': 'Stephen Sturgeon',
  'page_name': 'Finding Articles',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'American Periodical series',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=https://search.proquest.com/americanperiodicals?accountid=9673',
  'asset_url_to_match': 'search.proquest.com/americanperiodicals?accountid=9673',
  'box_name': 'Databases from Other Disciplines',
  'box_url': 'https://libguides.bc.edu/aframlit/articles#s-lg-box-1716921',
  'guide_id': '44042',
  'guide_name': 'Africa

...

  'box_name': 'Suggested Journal Databases',
  'box_url': 'https://libguides.bc.edu/swimmigration/journals#s-lg-box-842589',
  'guide_id': '44583',
  'guide_name': 'Social Work: Immigration & Refugees',
  'guide_owner': 'Adam Williams',
  'page_name': 'Journal Articles',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Made in L.A. (video)',
  'asset_url_raw': 'https://proxy.bc.edu/login?url=https://fod.infobase.com/PortalPlaylists.aspx?wID=95406&xtid=49785',
  'asset_url_to_match': 'fod.infobase.com/portalplaylists.aspx?wid=95406&xtid=49785',
  'box_name': 'Selected video clips',
  'box_url': 'https://libguides.bc.edu/swimmigration/film#s-lg-box-842747',
  'guide_id': '44583',
  'guide_name': 'Social Work: Immigration & Refugees',
  'guide_owner': 'Adam Williams',
  'page_name': 'Film & Video',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Ties that Bind: Immigration Stories (video)',
  'asset_url_raw': 'https://proxy.bc.edu/login?url=https://fod.infobase.com/PortalPlaylists.as

  'guide_owner': 'Margaret Cohen',
  'page_name': 'Biographies',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Oxford Dictionary of National Biography',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.oxforddnb.com/subscribed/',
  'asset_url_to_match': 'www.oxforddnb.com/subscribed',
  'box_name': 'Biographies of Educators',
  'box_url': 'https://libguides.bc.edu/c.php?g=108078&p=700471#s-lg-box-2082705',
  'guide_id': '108078',
  'guide_name': 'Doctoral Students Library Guide',
  'guide_owner': 'Margaret Cohen',
  'page_name': 'Biographies',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'ATLA Religion Database',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://search.ebscohost.com/login.asp?profile=religion&defaultdb=rfh',
  'asset_url_to_match': 'search.ebscohost.com/login.asp?profile=religion&defaultdb=rfh',
  'box_name': 'Education & Religion',
  'box_url': 'https://libguides.bc.edu/c.php?g=108078&p=700463#s-lg-box-2082696',
  'guide_id': '108078',
  '

 {'asset_name': 'PolicyFile',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.policyfile.com/search/initSearch.do',
  'asset_url_to_match': 'www.policyfile.com/search/initsearch.do',
  'box_name': 'General Sources',
  'box_url': 'https://libguides.bc.edu/communityresearch/stats#s-lg-box-4121256',
  'guide_id': '200899',
  'guide_name': 'Community Research Seminar',
  'guide_owner': 'Hannah Ha',
  'page_name': 'Find Statistics',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'ATLA Religion Database',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://search.ebscohost.com/login.aspx?authtype=ip,uid&profile=advance&defaultdb=rfh',
  'asset_url_to_match': 'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=advance&defaultdb=rfh',
  'box_name': 'Major Non Literary Databases',
  'box_url': 'https://libguides.bc.edu/nugent/literaturearticles#s-lg-box-4158972',
  'guide_id': '208412',
  'guide_name': 'ENGL333901: Digital Text, Material Image',
  'guide_owner': 'Steph

  'box_url': 'https://libguides.bc.edu/writing-the-past/news-archives#s-lg-box-9326575',
  'guide_id': '444634',
  'guide_name': 'EN453901: Writing the Past',
  'guide_owner': 'Stephen Sturgeon',
  'page_name': 'Newspapers, Magazines, & Archives',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Nineteenth Century Collections Online',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://find.galegroup.com/menu/start?userGroupName=mlin_m_bostcoll&prod=NCCO',
  'asset_url_to_match': 'find.galegroup.com/menu/start?usergroupname=mlin_m_bostcoll&prod=ncco',
  'box_name': 'Magazines & Periodicals',
  'box_url': 'https://libguides.bc.edu/writing-the-past/news-archives#s-lg-box-9326575',
  'guide_id': '444634',
  'guide_name': 'EN453901: Writing the Past',
  'guide_owner': 'Stephen Sturgeon',
  'page_name': 'Newspapers, Magazines, & Archives',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Nineteenth Century Masterfile',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://proxy.

  'page_name': 'Finding Test Reviews',
  'reason': 'uses proxy prefix'},
 {'asset_name': 'Hey Girl, Am I More than My Hair?: African American Women and '
                'Their Struggles with Beauty, Body Image, and Hair. (BC '
                'Access)',
  'asset_url_raw': 'https://proxy.bc.edu/login?url=https://search-ebscohost-com.proxy.bc.edu/login.aspx?direct=true&db=ehh&AN=22056097&site=ehost-live',
  'asset_url_to_match': 'search-ebscohost-com.proxy.bc.edu/login.aspx?direct=true&db=ehh&an=22056097&site=ehost-live',
  'box_name': 'Politics of Black Hair',
  'box_url': 'https://libguides.bc.edu/c.php?g=810112&p=5781735#s-lg-box-18383129',
  'guide_id': '810112',
  'guide_name': "Black Hair in Children's Literature",
  'guide_owner': 'Tiffeni Fontno',
  'page_name': 'Articles, Posts, Papers & Books',
  'reason': 'uses proxy prefix'}]


In [167]:
pprint.pprint(suspicious_asset_list)

[{'asset_name': 'Amadeus',
  'asset_url_raw': 'https://amadeus.bvdinfo.com/ip',
  'asset_url_to_match': 'amadeus.bvdinfo.com/ip',
  'box_name': 'New Databases',
  'box_url': 'https://libguides.bc.edu/companyinfotest/home#s-lg-box-438859',
  'guide_id': '24955',
  'guide_name': 'Company Info Test',
  'guide_owner': 'Jesse Martinez',
  'page_name': 'Home',
  'reason': 'matches full database url'},
 {'asset_name': 'Statista',
  'asset_url_raw': 'http://www.statista.com',
  'asset_url_to_match': 'www.statista.com',
  'box_name': 'New Databases',
  'box_url': 'https://libguides.bc.edu/companyinfotest/home#s-lg-box-438859',
  'guide_id': '24955',
  'guide_name': 'Company Info Test',
  'guide_owner': 'Jesse Martinez',
  'page_name': 'Home',
  'reason': 'matches full database url'},
 {'asset_name': 'Amadeus',
  'asset_url_raw': 'https://amadeus.bvdinfo.com/ip',
  'asset_url_to_match': 'amadeus.bvdinfo.com/ip',
  'box_name': 'New Databases',
  'box_url': 'https://libguides.bc.edu/companyinfotes

  'guide_id': '43965',
  'guide_name': 'Literary Research',
  'guide_owner': 'Stephen Sturgeon',
  'page_name': 'Secondary Criticism: Articles',
  'reason': 'matches full database url'},
 {'asset_name': 'Readers Guide Retrospective',
  'asset_url_raw': 'http://search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=rgr',
  'asset_url_to_match': 'search.ebscohost.com/login.aspx?authtype=ip,uid&profile=ehost&defaultdb=rgr',
  'box_name': 'Magazine/Popular Periodical Databases',
  'box_url': 'https://libguides.bc.edu/litresearch/secondaryarticles#s-lg-box-839919',
  'guide_id': '43965',
  'guide_name': 'Literary Research',
  'guide_owner': 'Stephen Sturgeon',
  'page_name': 'Secondary Criticism: Articles',
  'reason': 'matches full database url'},
 {'asset_name': 'OpinionArchives',
  'asset_url_raw': 'http://metasearch.opinionarchives.com',
  'asset_url_to_match': 'metasearch.opinionarchives.com',
  'box_name': 'Magazine/Popular Periodical Databases',
  'box_url': 'https:/

  'box_url': 'https://libguides.bc.edu/economics/govdata#s-lg-box-832656',
  'guide_id': '43967',
  'guide_name': 'Economics',
  'guide_owner': 'Barbara Mento',
  'page_name': 'Government Data',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Global Insight - Latest data will be found on this vendor '
                'interface(Formerly DRI/Basic Economic Database)',
  'asset_url_raw': 'http://myinsight.globalinsight.com/servlet/cats?pageContent=home&imask=1',
  'asset_url_to_match': 'myinsight.globalinsight.com',
  'box_name': 'Industry/Economic',
  'box_url': 'https://libguides.bc.edu/economics/financialdata#s-lg-box-833772',
  'guide_id': '43967',
  'guide_name': 'Economics',
  'guide_owner': 'Barbara Mento',
  'page_name': 'Financial Data',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Mergent Online',
  'asset_url_raw': 'http://www.mergentonline.com/compsearch.asp',
  'asset_url_to_match': 'mergentonline.com',
  'box_name': 'Company Statistics',
 

  'asset_url_raw': 'http://muse.jhu.edu/journals/lion_and_the_unicorn/',
  'asset_url_to_match': 'muse.jhu.edu',
  'box_name': 'Book Reviews/Databases',
  'box_url': 'https://libguides.bc.edu/c.php?g=43969&p=279272#s-lg-box-832662',
  'guide_id': '43969',
  'guide_name': "Children's Literature",
  'guide_owner': 'Margaret Cohen',
  'page_name': 'Book Reviews/Databases',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Literature Resource Center',
  'asset_url_raw': 'http://galenet.galegroup.com/servlet/LitRC;jsessionid=3B7D97180B2B5E65A15C44D4D7ED678E?locID=mlin_m_bostcoll',
  'asset_url_to_match': 'galenet.galegroup.com',
  'box_name': 'Book Reviews/Databases',
  'box_url': 'https://libguides.bc.edu/c.php?g=43969&p=279272#s-lg-box-832662',
  'guide_id': '43969',
  'guide_name': "Children's Literature",
  'guide_owner': 'Margaret Cohen',
  'page_name': 'Book Reviews/Databases',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Novelist k-8',
  'asset_url_ra

  'reason': 'matches full database url'},
 {'asset_name': 'North American Immigrant Letters, Diaries and Oral Histories',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://imld.alexanderstreet.com/',
  'asset_url_to_match': 'imld.alexanderstreet.com',
  'box_name': 'Personal Narrative Databases',
  'box_url': 'https://libguides.bc.edu/c.php?g=43974&p=279304#s-lg-box-836899',
  'guide_id': '43974',
  'guide_name': 'Irish History (Irish Studies)',
  'guide_owner': 'Kathy Williams',
  'page_name': 'Primary Sources',
  'reason': 'matches full database url'},
 {'asset_name': 'Oral History Online',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://asp6new.alexanderstreet.com/orhi/',
  'asset_url_to_match': 'asp6new.alexanderstreet.com/orhi',
  'box_name': 'Personal Narrative Databases',
  'box_url': 'https://libguides.bc.edu/c.php?g=43974&p=279304#s-lg-box-836899',
  'guide_id': '43974',
  'guide_name': 'Irish History (Irish Studies)',
  'guide_owner': 'Kathy Williams',
  'page

...

  'box_name': 'Education and Minorities',
  'box_url': 'https://libguides.bc.edu/lawandeducationreform/minorities#s-lg-box-4373735',
  'guide_id': '218556',
  'guide_name': 'Law and Education Reform (ELHE760901)',
  'guide_owner': 'Margaret Cohen',
  'page_name': 'Education and Minorities',
  'reason': 'matches a database domain name'},
 {'asset_name': 'ASSIA',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=https://search.proquest.com/assia/advanced?',
  'asset_url_to_match': 'search.proquest.com/assia/advanced?',
  'box_name': 'Other Social Science Databases',
  'box_url': 'https://libguides.bc.edu/lawandeducationreform/socialscience#s-lg-box-4373737',
  'guide_id': '218556',
  'guide_name': 'Law and Education Reform (ELHE760901)',
  'guide_owner': 'Margaret Cohen',
  'page_name': 'Other Social Science Databases',
  'reason': 'matches full database url'},
 {'asset_name': 'Humanities & Social Sciences Index Retrospective',
  'asset_url_raw': 'http://search.ebscohost.com/login.aspx?a

  'reason': 'matches a database domain name'},
 {'asset_name': 'INSPEC (1969 - )',
  'asset_url_raw': 'http://isiknowledge.com/inspec',
  'asset_url_to_match': 'isiknowledge.com/inspec',
  'box_name': 'The Science',
  'box_url': 'https://libguides.bc.edu/c.php?g=363767&p=2457710#s-lg-box-7492857',
  'guide_id': '363767',
  'guide_name': 'Energy Research',
  'guide_owner': 'Sally Wyman',
  'page_name': 'Find Journal Articles',
  'reason': 'matches full database url'},
 {'asset_name': 'OECD iLibrary: Organization for Economic Co-operation and '
                'Development',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.oecd-ilibrary.org/',
  'asset_url_to_match': 'www.oecd-ilibrary.org',
  'box_name': 'The Policy, the Politics and the Business',
  'box_url': 'https://libguides.bc.edu/c.php?g=363767&p=2457710#s-lg-box-7492859',
  'guide_id': '363767',
  'guide_name': 'Energy Research',
  'guide_owner': 'Sally Wyman',
  'page_name': 'Find Journal Articles',
  'reason': 'mat

  'guide_name': 'Making It Irish: Cultural Revival and Revolution',
  'guide_owner': 'Kathy Williams',
  'page_name': 'Locating Articles: Multidisciplinary',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Project Muse',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://muse.jhu.edu/',
  'asset_url_to_match': 'muse.jhu.edu',
  'box_name': 'Useful Multidisciplinary Databases',
  'box_url': 'https://libguides.bc.edu/c.php?g=454406&p=3103926#s-lg-box-9553921',
  'guide_id': '454406',
  'guide_name': 'Making It Irish: Cultural Revival and Revolution',
  'guide_owner': 'Kathy Williams',
  'page_name': 'Locating Articles: Multidisciplinary',
  'reason': 'matches full database url'},
 {'asset_name': 'JSTOR',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.jstor.org',
  'asset_url_to_match': 'jstor.org',
  'box_name': 'Searching JSTOR for Articles about Irish Arts and Crafts',
  'box_url': 'https://libguides.bc.edu/c.php?g=454406&p=3103932#s-lg-box-4331810',

  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://glossary.agiweb.org/dbtw-wpd/glossary/search.aspx',
  'asset_url_to_match': 'glossary.agiweb.org',
  'box_name': 'Glossaries&Dictionaries',
  'box_url': 'https://libguides.bc.edu/EESC3380/reference#s-lg-box-13446301',
  'guide_id': '620890',
  'guide_name': 'EESC 3380: Environmental Oceanography',
  'guide_owner': 'Enid Karr',
  'page_name': 'Encyclopedias & Directories',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Encyclopedia of Paleoclimatology and Ancient Environments)',
  'asset_url_raw': 'http://proxy.bc.edu/login?url=http://www.springerlink.com/content/m46752/?p=7d191003aafb4d94ad4ac28c09231d28&pi=0',
  'asset_url_to_match': 'springerlink.com',
  'box_name': 'Encyclopedias&Directories',
  'box_url': 'https://libguides.bc.edu/EESC3380/reference#s-lg-box-13446302',
  'guide_id': '620890',
  'guide_name': 'EESC 3380: Environmental Oceanography',
  'guide_owner': 'Enid Karr',
  'page_name': 'Encyclopedias &

  'guide_owner': 'Enid Karr',
  'page_name': 'Find Articles',
  'reason': 'matches a database domain name'},
 {'asset_name': 'Nature Reports: Climate Change',
  'asset_url_raw': 'http://www.nature.com/climate/index.html',
  'asset_url_to_match': 'nature.com',
  'box_name': 'Climate and Climate Change',
  'box_url': 'https://libguides.bc.edu/ENVS4941/topics#s-lg-box-16469295',
  'guide_id': '731683',
  'guide_name': 'ENVS 4941  Environmental Studies Senior Research Seminar',
  'guide_owner': 'Enid Karr',
  'page_name': 'By Topic',
  'reason': 'matches a database domain name'},
 {'asset_name': '<b>ProQuest Historical U.S. Newspapers</b>',
  'asset_url_raw': 'https://search.proquest.com/advanced?accountid=9673&selectids=1005629,1007871,1007272,1006744,1008864,1007155,1007154,1006359,1005877,1006360,1006442',
  'asset_url_to_match': 'search.proquest.com',
  'box_name': 'Historical Newspapers',
  'box_url': 'https://libguides.bc.edu/c.php?g=738513&p=5280986#s-lg-box-16646194',
  'guide_id':

In [168]:
# convert suspicious_asset_list to a csv file
# https://stackoverflow.com/questions/3086973/how-do-i-convert-this-list-of-dictionaries-to-a-csv-file
fieldnames = [
    "guide_id", "guide_name", "guide_url", "guide_owner", 
    "page_id", "page_name", "page_url", 
    "box_id", "box_name", "box_url", 
    "asset_id", "asset_name", "asset_url_raw", "asset_url_to_match"
]
timestr = time.strftime("%Y%m%d-%H%M%S")
csv_filename = 'suspicious_assets-%s-%s.csv' % (db_dump_file_no_extention, timestr)
keys = suspicious_asset_list[0].keys()
with open(csv_filename, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(suspicious_asset_list)